### Ballpark Factor

Here we aim to develop a ratio between the total number of runs scored in games played at a given team's home ballpark compared to the total number of runs scored in games played at away ballparks. This ratio will allow us to adjust our individual statistics based on any competetive advantages given by a players home ballpark.

Before we begin, import the necessary statements and create our list of teams with their corresponding numbers on the DakStats website.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pickle

In [3]:
teams =["Bethel", "Goshen", "Grace", "HU", "IWU", "Marian", "MVNU", "SAU", "SFU", "Taylor"]
t_nums = [1629, 1678, 1679, 1688, 1694, 1717, 1736, 1780, 1805, 1784]

First, we gather the different DakStats URL's for each team.

In [4]:
urls = ['http://www.dakstats.com/WebSync/Pages/Team/TeamSchedule.aspx?association=10&sg=MBA&sea=NAIMBA_2019&team=' +
        str(num) for num in t_nums]
#Create a handle, page, to handle the contents of the website
pages = [requests.get(url) for url in urls]
#Store the page as an element tree using BeautifulSoup4
soups = [BeautifulSoup(page.content) for page in pages]

The code below collects all of the html tables from the different teams' webpages on DakStats.

In [5]:
team_tables = [
  [
    [
      [td.get_text(strip=True) for td in tr.find_all('td')] 
      for tr in table.find_all('tr') 
    ]#for each row in each table
    for table in soup.find_all('table') 
  ]#for each table on each webpage
  for soup in soups 
]#for each team's webpage

This loop allows us to locate the table from the webpage that contains the data we are interested in. We find the headers in the 33rd table and the actual data in the 35th table. We will assume that this is the same for all teams.

In [6]:
for i in range(len(team_tables[2])):
  #print(i, team_tables[2][i])
  #The line ablve is commented out becuause we only needed to run it once to find the location of the data on the webpage.
  pass

Next, we define the column names for our dataframe.

In [7]:
headers = [['Date', 'Opponent', 'Location', 'Score', 'Outcome'] for tables in team_tables]
headers[2]

['Date', 'Opponent', 'Location', 'Score', 'Outcome']

Here, we collected the data into the list `team_rows`. We used the code `[:5]` to take only the first 5 columns of data and we used the code `[1::2]` to collect the data from every other row, since between each list of data there is an empty list.

In [8]:
team_rows = [[r[:5] for r in tables[35][1::2]] for tables in team_tables]
team_rows[2][:9]

[['2/27/2019', 'Lourdes (Ohio)', 'N', '3-4', 'L'],
 ['2/27/2019', 'Lourdes (Ohio)', 'N', '4-8', 'L'],
 ['3/2/2019', 'Cornerstone (Mich.)', 'N', '3-4', 'L'],
 ['3/2/2019', 'Trinity Baptist', 'N', '5-1', 'W'],
 ['3/4/2019', 'Michigan-Dearborn', 'N', '13-1', 'W'],
 ['3/5/2019', 'Rochester (Mich.)', 'N', '24-4', 'W'],
 ['3/6/2019', 'Robert Morris (Ill.)', 'N', '10-9', 'W'],
 ['3/8/2019', 'Bethel (Ind.) *', 'N', '13-6', 'W'],
 ['3/9/2019', 'Bethel (Ind.) *', 'N', '14-2', 'W']]

Now, we put the data into a dataframe.

In [9]:
dfc = [pd.DataFrame(columns = headers[i], data = team_rows[i]) for i in range(len(headers))]
dfc[2][:5]

,Date,Opponent,Location,Score,Outcome
0,2/27/2019,Lourdes (Ohio),N,3-4,L
1,2/27/2019,Lourdes (Ohio),N,4-8,L
2,3/2/2019,Cornerstone (Mich.),N,3-4,L
3,3/2/2019,Trinity Baptist,N,5-1,W
4,3/4/2019,Michigan-Dearborn,N,13-1,W


We subset the data to only include opponents with an asterisk which denotes conference games.

In [10]:
conf_df = [df[df.Opponent.str.contains("*", regex = False)] for df in dfc]
conf_df[2][:5]

,Date,Opponent,Location,Score,Outcome
7,3/8/2019,Bethel (Ind.) *,N,13-6,W
8,3/9/2019,Bethel (Ind.) *,N,14-2,W
9,3/9/2019,Bethel (Ind.) *,N,3-1,W
10,3/14/2019,Taylor (Ind.) *,A,5-15,L
11,3/16/2019,Taylor (Ind.) *,A,2-10,L


The below code copies the dataframe with `.copy()` to avoid errors, splits the "Score" column into two columns, one for the selected team and one for the opponent. Then, the code `str.replace(' \*', '', regex= True)` eliminates the parentheses and the number between them for extra-inning games.

In [11]:
tidy_conf = conf_df.copy()
for i, df in enumerate(conf_df):
  split_scores = df['Score'].str.replace(r"\(.*\)","").str.split('-', expand = True)
  tidy_conf[i] = df.assign(Score = pd.to_numeric(split_scores[0]),
                           Opp_score = pd.to_numeric(split_scores[1]),
                           Opponent = df.Opponent.str.replace(' \*', '', regex= True),
                           Date = pd.to_datetime(df.Date)
                           )
tidy_conf[2][:5]

,Date,Opponent,Location,Score,Outcome,Opp_score
7,2019-03-08,Bethel (Ind.),N,13,W,6
8,2019-03-09,Bethel (Ind.),N,14,W,2
9,2019-03-09,Bethel (Ind.),N,3,W,1
10,2019-03-14,Taylor (Ind.),A,5,L,15
11,2019-03-16,Taylor (Ind.),A,2,L,10


Finally, we subset the home games in one dataframe and the away games in another, and then use the sum of the two score columns in each to calculate our park factor for each team.

In [12]:
conf_h = [df[df.Location.str.contains("H", regex = False)] for df in tidy_conf]
conf_a = [df[df.Location.str.contains("A", regex = False)] for df in tidy_conf]

h_runs_per_game = [(df.Score.sum() + df.Opp_score.sum())/len(df.index) for df in conf_h]
a_runs_per_game = [(df.Score.sum() + df.Opp_score.sum())/len(df.index) for df in conf_a]
park_factor = [h_runs_per_game[i]/a_runs_per_game[i] for i in range(len(headers))]
park_factor

[0.963768115942029,
 1.1224268689057422,
 0.8870967741935484,
 0.7320261437908496,
 1.0357675111773472,
 0.7938931297709924,
 0.8571428571428571,
 1.15,
 1.3483365949119375,
 1.3363844393592679]